This Kernel is my attempt to apply FastAI tabular implementation . The dataset is basically asking to to predict if the customer with a given set of features will cancel or not cancel . This is not a competition dataset so I had to select a evaluation metric. 

Seeing the un-balanced nature of the dependent variable i choose auc-roc-score which is a binary metric designed for unbalanced dataset. So basically the auc-roc-score says how confident a model is between the possible outcome . Here my auc-roc-socre of 0.948 says the model is 94.8 percent certain in predicting 1 vs 0 

I have included some EDA in the below kernel . Please go through the kernel . Please upvote if you like it and feel free to comment!!!

In [ ]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.figure_factory import create_table
import plotly.express as px

plt.style.use('fivethirtyeight')

import warnings 
warnings.filterwarnings('ignore')


In [ ]:
df=pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')

In [ ]:
df.head()

 So the job here is to predict whether the booking goes through or not that is is_canceled column is our dependent variable . <br>
 
 This model could be very beneficial for the hotel managment to plan inventory or can also help better schedule worker's schedule <br>

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['required_car_parking_spaces'].unique()

In [ ]:
df['adr'].unique()

In [ ]:
df['reserved_room_type'].value_counts()

In [ ]:
df['assigned_room_type'].value_counts()

In [ ]:
df['country'].unique()

In [ ]:
df['booking_changes'].unique()

In [ ]:
df['distribution_channel'].unique()

In [ ]:
df['is_repeated_guest'].value_counts()

In [ ]:
df['deposit_type'].value_counts()

In [ ]:
df['adults'].unique()

In [ ]:
df['children'].unique()

In [ ]:
df['babies'].unique()

In [ ]:
df['previous_cancellations'].unique()

In [ ]:
df['meal'].unique()

In [ ]:
df['total_of_special_requests'].value_counts()

In [ ]:
df['customer_type'].value_counts()

# Lets Explore the dataset with some prelimanary EDA 

In [ ]:
color=sns.color_palette()[1]
sns.countplot(data=df,x='is_canceled',color=color);
plt.title('Distribution of Cancelations')

As the data skewed we can chose Area Under the ROC cuve/F1 score / Recall/Precision as the metrics we will chose ROC curve as our guiding metric for this problem 

In [ ]:
color=sns.color_palette()[1]
sns.countplot(data=df,x='hotel',color=color);
plt.title('Hotel Type Count')

In [ ]:
df.shape[0]

In [ ]:
sns.set_style('whitegrid')
row=df.shape[0]
color=sns.color_palette()[0]
sns.countplot(data=df,x='is_canceled',hue='hotel',color=color);
#df.plot(kind='bar',stacked=True,legend=False)
plt.yticks([10000,20000,30000,40000],[8,17,25,34]);
plt.ylabel('percentage');
plt.title('Cancellation count in Hotel vs Resort');

In [ ]:
df['arrival_date_year'].min(),df['arrival_date_year'].max()

In [ ]:
fig = px.bar(x=df["arrival_date_year"].value_counts().index, 
             y=(df.groupby('arrival_date_year').aggregate('sum')['is_canceled']/df.groupby('arrival_date_year').aggregate('count')['is_canceled'])*100,
             
             )

fig.update_layout(
    title={
        'text': "Percent Cancellation vs Year",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Year",
    yaxis_title="Percent Cancellation",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ))

fig.update_xaxes(tickvals=[2015,2016,2017])
fig.show()

The percentage of booking cancelation seem pretty constant across year hence it makes more sense to better predict cancellation about a booking 

# Timelaps of cancellations 

In [ ]:
timelapse=df.groupby(['arrival_date_year','arrival_date_month','country']).aggregate('sum')['is_canceled'].to_frame().reset_index()
timelapse['month-year']=timelapse.arrival_date_month.astype(str)+' '+timelapse.arrival_date_year.astype(str)

px.choropleth(timelapse, locations="country", color='is_canceled', hover_name="country", animation_frame='month-year',
              color_continuous_scale=px.colors.sequential.Plasma, projection="natural earth")

We see that most countries fall pretty on the lower range of the scale apart from countries close along the Meditarrian Belt such as Portugal , Spain , UK Italy and France so this is one the variable to look forward too 

# Cancelations by month 


In [ ]:
#df_month=df.groupby('arrival_date_month').aggregate('sum').reset_index()

# Got the below values from the above groupby dataframe 

month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']

value=[1807,2696,3149,4524,4677,4535,4742,5239,4116,4246,2122,2371]

fig=px.line(x=month,y=value)
fig.show()

From the above graph we see that the cancellation increases as the year progress reaching the peak at the middle of the year . Let see the percentages to get a much more clear picture as this could also be due the number of booking reaching a high during the middle of summer 

In [ ]:
df_percent=df.groupby('arrival_date_month').aggregate('mean').reset_index()

#Values taken from above dataframe 

month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']

value=[30.4,33.4,32.15,40.79,39.66,41.45,37.45,37.75,39.17,38.04,31.23,34.97]

fig=px.line(x=month,y=value)
fig.show()

We do indeed see a higher cancelations in summer months 

In [ ]:
deposit=df.groupby('deposit_type').aggregate('sum')['is_canceled'].to_frame().reset_index()

fig=(px.pie(deposit, values=deposit['is_canceled'], names='deposit_type',color='deposit_type', color_discrete_map=
                                {'Non Refund':'gold',
                                 'No Deposit':'darkorange',
                                 'Refundable':'lightgreen'
                                 }
                            ))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

The above graph shows the distribution of cancelation among the various deposit types . With No deposit leading the cancellations when we consider just the cancelation <br>

But the other way to look at this would be the what percent of each booking type canceled that is how many people using the non-refund booking types cancelled doing that actually presents us a completely different picture . I always prefer percentage as compared to absolutes when the counts are different <br>

In [ ]:
deposit=df.groupby('deposit_type').mean()['is_canceled'].to_frame().reset_index()


fig=(px.pie(deposit, values=deposit['is_canceled'], names='deposit_type',color='deposit_type', color_discrete_map=
                                {'Non Refund':'gold',
                                 'No Deposit':'darkorange',
                                 'Refundable':'lightgreen'
                                 }
                                
                                   ))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


Seeing the corelation value between difference in room type booked and assigned we see a weakish negative corelation -0.24 so this feature should be included in our analysis 

In [ ]:
plt.figure(figsize=(20,20));
sns.heatmap(df.corr(),annot=True,linewidths=0.1,cbar=False);


From the above heatmap most variables seem to have either a decent positive or negative relationship with the dependent variable 

In [ ]:
#Lets compute a new column in the above dataset to compute number of days stay booked in the hotel 

df['stays_total_duration']=df['stays_in_week_nights']+df['stays_in_weekend_nights']

In [ ]:
plt.figure(figsize=(15,8))

plt.subplot(1,2,1)
sns.boxplot(x=df['babies'],y=df['stays_total_duration'],hue=df['is_canceled']);


plt.subplot(1,2,2)
sns.boxplot(x=df['children'],y=df['stays_total_duration'],hue=df['is_canceled']);

We see same trend for people having babies and children and their cancelations with weekend and weekday stays . 

* Among the people who who have 0 childredn or babies there is not much difference . <br>
* But people who cancel with 1 or 2 babies usually have a longer stay booked at the hotel . <br>
* Greater than 2 babies count would be do not have any cancelations as this could be group tours and hence cancelations do not depend on single per's will<br>

In [ ]:
plt.figure(figsize=(10,10)),
sns.boxplot(x=df['adults'],y=df['stays_total_duration'],hue=df['is_canceled']);

Adults do not have the same distribution of cancelation vs total duration 

In [ ]:
plt.figure(figsize=(15,8))

plt.subplot(1,2,1)
sns.boxplot(x=df['customer_type'],y=df['stays_total_duration'],hue=df['is_canceled']);

plt.subplot(1,2,2)
sns.countplot(x=df['customer_type'],hue=df['is_canceled']);



Here we can see that most customer types is concentrated in Transient and Transient-Party 

Also among the customer_type vs Stays_total_duration we see that group type customer who cancel have a higher range in days of stays

# Key Observations 

* Cancallation increases as the year progresses with the cancellation peaking at the end of year 
* The countries which dominate cancellation are in the neighbourhood of Spain and Portugal in Europe
* From heatmap we have decent corelation with most variables 
* Among the cancallation vs deposits types  we see that No-deposit have higher percentage of cancellation 
* Also among the customer_type vs Stays_total_duration we see that group type customer who cancel have a higher range in days of stays
* Among the people who who have 0 childredn or babies there is not much difference . <br>
* But people who cancel with 1 or 2 babies usually have a longer stay booked at the hotel . <br>
* Greater than 2 babies count would be do not have any cancelations as this could be group tours and hence cancelations do not depend on single per's will<br>

# Implementing Models 

In [ ]:

from fastai.tabular import * 

In [ ]:
df.shape

In [ ]:
# Lets train the model for 2015 and 2016 and validate it for 2017 

df_train=df[df['arrival_date_year']!=2017]
df_test=df[df['arrival_date_year']==2017]

In [ ]:
# Classify the variables into response , catergorical and numerical 

response='is_canceled'

catergorical=['hotel','meal','country','market_segment','customer_type','deposit_type','assigned_room_type','reserved_room_type','is_repeated_guest'\
              ,'distribution_channel' ,'arrival_date_year','arrival_date_month','arrival_date_week_number','arrival_date_day_of_month']

numerical=['adults','children','babies','stays_total_duration','stays_in_weekend_nights','stays_in_week_nights','required_car_parking_spaces'\
          ,'total_of_special_requests','booking_changes','previous_bookings_not_canceled','previous_cancellations','days_in_waiting_list']

procs = [FillMissing, Categorify, Normalize]

path='/kaggle/working'

In [ ]:
data = (TabularList.from_df(df_train, path=path, cat_names=catergorical, cont_names=numerical, procs=procs)
                           .split_by_rand_pct(0.15)
                           .label_from_df(cols=response)
                           .add_test(df_test)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

In [ ]:
learn = tabular_learner(data, layers=[200,100], metrics=[AUROC()])

In [ ]:
learn.fit(8, 1e-2)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-3, wd=0.2)

In [ ]:
learn.fit_one_cycle(5, 1e-4)

In [ ]:
learn.fit_one_cycle(5, 1e-5, wd=0.2)

In [ ]:
learn.save('Hotel_Cancellation')